<a href="https://colab.research.google.com/github/jkim1134/repository-bioinformatics/blob/main/Colab_Qiime2repository.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3

"""Set up Qiime 2 on Google colab.

Do not use this on o local machine, especially not as an admin!
"""

import os
import sys
import shutil
from subprocess import Popen, PIPE, run

r = Popen(["pip", "install", "rich"])
r.wait()
from rich.console import Console  # noqa
con = Console()

PREFIX = "/usr/local/miniforge3/"

has_conda = "conda version" in os.popen("%s/bin/conda info" % PREFIX).read()
qiime_installed = os.path.exists(os.path.join(PREFIX, "envs", "qiime2", "bin", "qiime"))
qiime_active = "QIIME 2 release:" in os.popen("qiime info").read()


MINICONDA_PATH = (
    "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh"
)

QIIME_YAML_TEMPLATE = (
    "https://data.qiime2.org/distro/amplicon/qiime2-amplicon-{version}-py{python}-linux-conda.yml"
)

if len(sys.argv) == 2:
    version = sys.argv[1]
else:
    version = "2025.4"

qiime_ver = tuple(int(v) for v in version.split("."))

if qiime_ver < (2021, 4):
    pyver = "36"
elif qiime_ver < (2024, 5):
    pyver = "38"
elif qiime_ver < (2024, 10):
    pyver = "39"
else:
  pyver = "310"

CONDA = "mamba"
CONDA_ARGS = ["-q"] if CONDA == "conda" else ["-y"]

if tuple(float(v) for v in version.split(".")) < (2023, 9):
    QIIME_YAML_TEMPLATE = (
        "https://data.qiime2.org/distro/core/qiime2-{version}-py{python}-linux-conda.yml"
    )

QIIME_YAML_URL = QIIME_YAML_TEMPLATE.format(version=version, python=pyver)
QIIME_YAML = os.path.basename(QIIME_YAML_URL)


def cleanup():
    """Remove downloaded files."""
    if os.path.exists(os.path.basename(MINICONDA_PATH)):
        os.remove(os.path.basename(MINICONDA_PATH))
    if os.path.exists(QIIME_YAML):
        os.remove(QIIME_YAML)
    if os.path.exists("/content/sample_data"):
        shutil.rmtree("/content/sample_data")
    con.log(":broom: Cleaned up unneeded files.")


def run_and_check(args, check, message, failure, success, console=con):
    """Run a command and check that it worked."""
    console.log(message)
    r = Popen(args, env=os.environ, stdout=PIPE, stderr=PIPE,
              universal_newlines=True)
    o, e = r.communicate()
    out = o + e
    if r.returncode == 0 and check in out:
        console.log("[blue]%s[/blue]" % success)
    else:
        console.log("[red]%s[/red]" % failure, out)
        open("logs.txt", "w").write(out)
        cleanup()
        sys.exit(1)

def run_in_env(cmd, env, console=con):
    """Activate a conda environment in colab."""
    conda_profile = os.path.join(PREFIX, "etc", "profile.d", "conda.sh")
    console.log(f":snake: Activating the {env} environment.")

    full = f". {conda_profile} && conda activate {env} && {cmd}"
    return run(
        full,
        shell=True,
        executable="/bin/bash",
        capture_output=True,
        text=True
    )

def mock_qiime2(console=con):
    con.log(":penguin: Setting up the Qiime2 command...")
    conda_profile = os.path.join(PREFIX, "etc", "profile.d", "conda.sh")
    with open("/usr/local/bin/qiime", "w") as mocky:
        mocky.write("#!/usr/bin/env bash")
        mocky.write(f'\n\n. {conda_profile} && conda activate qiime2 && qiime "$@"\n')
    run("chmod +x /usr/local/bin/qiime", shell=True, executable="/bin/bash")
    con.log(":penguin: Done.")

if __name__ == "__main__":
    if not has_conda:
        run_and_check(
            ["wget", MINICONDA_PATH],
            "saved",
            ":snake: Downloading miniforge...",
            "failed downloading miniforge :sob:",
            ":snake: Done."
        )

        run_and_check(
            ["bash", os.path.basename(MINICONDA_PATH), "-bfp", PREFIX],
            "installation finished.",
            ":snake: Installing miniforge...",
            "could not install miniforge :sob:",
            ":snake: Installed miniforge to `/usr/local`."
        )
    else:
        con.log(":snake: Miniforge is already installed. Skipped.")

    if not qiime_installed:
        run_and_check(
            ["wget", QIIME_YAML_URL],
            "saved",
            ":mag: Downloading Qiime 2 package list...",
            "could not download package list :sob:",
            ":mag: Done."
        )

        if CONDA == "mamba":
            CONDA_ARGS.append("-y")

        run_and_check(
            [PREFIX + "bin/" + CONDA, "env", "create", *CONDA_ARGS, "-n", "qiime2", "--file", QIIME_YAML],
            "Verifying transaction: ...working... done" if CONDA == "conda" else "Transaction finished",
            f":mag: Installing Qiime 2 ({version}). This may take a little bit.\n :clock1:",
            "could not install Qiime 2 :sob:",
            ":mag: Done."
        )

        mock_qiime2()

        con.log(":evergreen_tree: Installing empress...")
        rc = run_in_env(
            "pip install --verbose Cython && pip install iow==1.0.7 empress",
            "qiime2"
        )
        if rc.returncode == 0:
            con.log(":evergreen_tree: Done.")
        else:
            con.log("could not install Empress :sob:")
    else:
        con.log(":mag: Qiime 2 is already installed. Skipped.")
        if not qiime_active:
            mock_qiime2()

    run_and_check(
        ["qiime", "info"],
        "QIIME 2 release:",
        ":bar_chart: Checking that Qiime 2 command line works...",
        "Qiime 2 command line does not seem to work :sob:",
        ":bar_chart: Qiime 2 command line looks good :tada:"
    )

    cleanup()

    con.log("[green]Everything is A-OK. "
            "You can start using Qiime 2 now :thumbs_up:[/green]")

In [ ]:
from google.colab import files
import os

print("📁 UPLOAD ONLY THE ZIP FILE:")
print("1. Click the 📁 FOLDER icon on the LEFT")
print("2. Click the 📤 UPLOAD button")
print("3. Select ONLY: fly_amplicon.zip")
print("4. Wait for upload to complete")

# Create project structure
!mkdir -p /content/fly_project/{data,results}

print("✅ Ready for zip file upload!")

In [ ]:
# Move zip to project folder
!mv /content/fly_amplicon.zip /content/fly_project/data

# Extract everything
!unzip /content/fly_project/data/fly_amplicon.zip -d /content/fly_project/data/

# Move files from subfolder to main data folder
!mv /content/fly_project/data/fly_amplicon/* /content/fly_project/data/
!rmdir /content/fly_project/data/fly_amplicon/

print("✅ Files extracted and organized!")
!ls -la /content/fly_project/data

In [ ]:
print("🚀 STARTING ANALYSIS...")

# 1. Import data
!qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path /content/fly_project/data/manifest.tsv \
  --output-path /content/fly_project/results/demux.qza \
  --input-format SingleEndFastqManifestPhred33V2

# 2. Quality control
!qiime demux summarize \
  --i-data /content/fly_project/results/demux.qza \
  --o-visualization /content/fly_project/results/demux.qzv

print("✅ STEP 1 COMPLETE!")

In [ ]:
print("🚀 RESTARTING ANALYSIS WITH FIXED PATHS...")

# 1. Import data
print("1. Importing FASTQ files...")
!qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path /content/fly_project/data/manifest.tsv \
  --output-path /content/fly_project/results/demux.qza \
  --input-format SingleEndFastqManifestPhred33V2

# 2. Quality control
print("2. Quality control...")
!qiime demux summarize \
  --i-data /content/fly_project/results/demux.qza \
  --o-visualization /content/fly_project/results/demux.qzv

print("✅ IMPORT SUCCESSFUL!")

In [ ]:
# 3. DADA2 Denoising - re-adding this step to complete the original sequence of operations
print("3. DADA2 Denoising...")
!qiime dada2 denoise-single \
      --i-demultiplexed-seqs /content/fly_project/results/demux.qza \
      --p-trim-left 0 \
      --p-trunc-len 250 \
      --o-representative-sequences /content/fly_project/results/rep-seqs.qza \
      --o-table /content/fly_project/results/table.qza \
      --o-denoising-stats /content/fly_project/results/denoising-stats.qza

print("✅ DADA2 complete!")

In [ ]:
print("4. Taxonomic Classification...")
!qiime feature-classifier classify-sklearn \
  --i-classifier /content/fly_project/data/classifier.qza \
  --i-reads /content/fly_project/results/rep-seqs.qza \
  --o-classification /content/fly_project/results/taxonomy.qza

print("✅ Taxonomy complete!")

In [ ]:
print("5. Creating Taxa Barplot...")
!qiime taxa barplot \
  --i-table /content/fly_project/results/table.qza \
  --i-taxonomy /content/fly_project/results/taxonomy.qza \
  --m-metadata-file /content/fly_project/data/metadata.tsv \
  --o-visualization /content/fly_project/results/taxa-bar-plots.qzv

print("✅ Barplot created!")

In [ ]:
print("🎉 ANALYSIS COMPLETE!")
print("\n📊 YOUR RESULTS:")
!ls -la /content/fly_project/results/

print("\n🌐 TO VIEW RESULTS:")
print("1. Download .qzv files from the results folder")
print("2. Go to: https://view.qiime2.org")
print("3. Drag and drop files to view interactive plots")
print("\n📈 Key visualizations:")
print("• demux.qzv - Quality control")
print("• taxa-bar-plots.qzv - MAIN RESULTS (microbes by state)")

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences /content/fly_project/results/rep-seqs.qza \
  --o-alignment /content/fly_project/results/aligned-rep-seqs.qza \
  --o-masked-alignment /content/fly_project/results/masked-aligned-rep-seqs.qza \
  --o-tree /content/fly_project/results/unrooted-tree.qza \
  --o-rooted-tree /content/fly_project/results/rooted-tree.qza

In [ ]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny /content/fly_project/results/rooted-tree.qza \
  --i-table /content/fly_project/results/table.qza \
  --p-sampling-depth 1103 \
  --m-metadata-file /content/fly_project/data/metadata.tsv \
  --output-dir /content/fly_project/results/diversity-core-metrics-phylogenetic

In [ ]:
# The previous command failed due to an issue with the metadata file.
# Let's inspect the metadata.tsv file to understand the problem.

print("Content of metadata.tsv:")
!cat /content/fly_project/data/metadata.tsv

print("6. Running Alpha Group Significance...")
!qiime diversity alpha-group-significance \
  --i-alpha-diversity /content/fly_project/results/diversity-core-metrics-phylogenetic/faith_pd_vector.qza \
  --m-metadata-file /content/fly_project/data/metadata.tsv \
  --o-visualization /content/fly_project/results/faith-pd-group-significance.qzv

!qiime diversity alpha-group-significance \
  --i-alpha-diversity /content/fly_project/results/diversity-core-metrics-phylogenetic/evenness_vector.qza \
  --m-metadata-file /content/fly_project/data/metadata.tsv \
  --o-visualization /content/fly_project/results/evenness-group-significance.qzv

print("✅ Alpha group significance analysis complete!")